In [8]:
import pandas as pd
import os
import pickle
import click



## create mloflow env
conda create -n <env-name>

conda create -n mlflow python=3.9

 conda activate mlflow

# To deactivate an active environment, use
 conda deactivate

In [3]:
## ensure the reqirements are installled
## use the terminal 
##! pip install -r requirements.txt

! pip install mlflow
import mlflow

## Notes 
mlfow.set_tracking_uri("sqlite://mflow.db")

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [22]:
!mlflow ui --backend-store-uri sqlite:///myflow.db

2024/06/07 19:37:48 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/06/07 19:37:48 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [23]:
mlflow.set_tracking_uri("sqlite:///myflow.db")
mlflow.set_experiment("sheyi-ml")


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2024/06/07 19:39:08 INFO mlflow.tracking.fluent: Experiment with name 'sheyi-ml' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/my_mlops_bootcamp/week-02/mlruns/1', creation_time=1717789148854, experiment_id='1', last_update_time=1717789148854, lifecycle_stage='active', name='sheyi-ml', tags={}>

## Preprocessing stage and getting the training data 

In [9]:
df = pd.read_parquet("./TAXI_DATA_FOLDER/green_tripdata_2023-01.parquet")

df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

categorical = ['PU_DO']
numerical = ['trip_distance']
dicts = df[categorical + numerical].to_dict(orient='records')


## Instead of above
Run the below rather for the preprocessing stage 
#### python preprocess_data.py --raw_data_path TAXI_DATA_FOLDER --dest_path ./output

In [24]:

df.head()
## Duration is the target variable for the 

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,...,0.5,4.03,0.0,None,1.0,24.18,1.0,1.0,2.75,11.016667
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,...,0.5,2.64,0.0,None,1.0,15.84,1.0,1.0,0.00,6.766667
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,...,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00,6.333333
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,...,1.5,1.70,0.0,None,1.0,10.20,1.0,1.0,0.00,5.816667
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,...,1.5,0.00,0.0,None,1.0,8.00,1.0,1.0,0.00,5.966667


## Train the dataset 
python train.py --data_path output

## Added these codes to see if the script in train .py ia=s working 
 print("The script is working ")
 print(rmse)

The script is working 
5.431162180141208

## command from the training video
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)